## Experiment-06
### Amirreza Fosoul and Bithiah Yuan

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
import string
import csv
import re
import time
from pyspark.sql.functions import *
from pyspark.sql.functions import split
import pyspark.sql.functions as f
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql.functions import desc
from pyspark.sql.window import Window as W
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import concat, col, lit, size
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import IntegerType
import random
spark = SparkSession.builder.appName('ex6').getOrCreate()
sc = spark.sparkContext

In [2]:
# Read user ratings into Dataframe
user_df = spark.read.option("delimiter", ";").csv('./example0.txt')
#user_df = spark.read.option("delimiter", ";").csv('./example.txt')
user_df = user_df.select(col("_c0").alias("userID"), col("_c1").alias("paperID"))

user_pre = user_df

user_df = user_df.select("userID", f.split("paperID", ",").alias("papers"), f.explode(f.split("paperID", ",")).alias("paperID"))
user_df = user_df.drop("papers")

# Count the number of distinct users
numUsers = user_df.select("userID").distinct().count()

# Get a dataframe of the distinct papers
d_paper = user_df.select("paperID").distinct()

userLib = user_pre.withColumn("paperID", split(col("paperID"), ",").cast(ArrayType(IntegerType()))).orderBy("userID")

get_len_udf = udf(lambda x: len(x), IntegerType())

userLib = userLib.withColumn("libSize", get_len_udf(col("paperID")))

userLib = userLib.select("userID", "libSize", f.explode("paperID").alias("paperID"))

In [28]:

# Read in records of paper information
#w_df = spark.read.csv('./papers.csv')
w_df = spark.read.csv('./paper0.csv')
#w_df = w_df.select("_c0", "_c01" "_c06", "_c09", "_c13", "_c14")
w_df = w_df.select("_c0", "_c1", "_c6", "_c9")
#w_df = w_df.select(col("_c0").alias("paperID"), col("_c01").alias("type"), col("_c06").alias("pages"), col("_c09").alias("year"), col("_c13").alias("title"), col("_c14").alias("abstract"))
w_df = w_df.select(col("_c0").alias("paperID"), col("_c1").alias("type"), col("_c6").alias("pages"), col("_c9").alias("year"))

#w_df.show()

### Exercise 6. 1 (Problem Modeling)

In [42]:
# Create a user hash map dataframe by adding a column with descending row numbers 
# to the pre-explode user dataframe
# user_hash_map = user_pre.withColumn("user_idx", F.monotonically_increasing_id().cast(IntegerType()))

# Select the user indices and paperID
#rating = user_hash_map.select('user_idx', 'paperID')
rating = userLib.select('userID', 'paperID', 'libSize')

# Split the string if all paperIDs into individual paperIDs and cast the datatype to int
rating = rating.withColumn("papers", split(col("paperID"), ",\s*").cast(ArrayType(IntegerType())).alias("papers"))
# Drop the column with the string of paperIDs
rating = rating.drop("paperID")

# Add a column of 1's called ratings for the rated papers w.r.t to the user index
rating = rating.withColumn("rating", lit(1))

# Transform the distinct paperIDs dataframe to a list
paper_list = list(d_paper.select('paperID').toPandas()['paperID'])
# Map each distinct paper into int
paper_list = list(map(int, paper_list))

# Function to call in udf
def unrated(papers):
    # Transform the list of distinct papers and the list of rated papers of each user to a set
    # Substract the two sets to get the list of unrated papers for each user
    # Transform back to list
    unrated = list(set(paper_list) - set(papers))

    # Select n random papers from the unrated papers where n is the 
    # length of the rated papers list of each user
    unrated_papers = random.sample(unrated, len(papers))
            
    return unrated_papers


# udf to get a list of unrated papers with the length of rated papers for each user
get_unrated = udf(lambda x: unrated(x), ArrayType(IntegerType()))

# Add a new column of unrated papers for each user
rating = rating.withColumn("unrated", get_unrated(rating.papers))

# Create a dataframe with the rated papers and their ratings of 1's
rated_df = rating.select("userID", "papers", "rating", "libSize")

# Create a dataframe with the unrated papers
unrated_df = rating.select("userID", col("unrated").alias("papers"), "libSize")
# Add a column of ratings of 0's to the unrated papers
unrated_df = unrated_df.withColumn("rating", lit(0))
unrated_df = unrated_df.select("userID", "papers", "rating", "libSize")

# Union the rated and unrated dataframes and order by the user indices
rating_matrix = rated_df.union(unrated_df).orderBy("userID")

# Explode the lists of paperIDs w.r.t. each user
rating_matrix = rating_matrix.select("userID", "libSize", f.explode("papers").alias("paperID"), 'rating')

feature_df = rating_matrix.join(w_df, ["paperID"]).orderBy("userID")

window = Window.partitionBy(col("userID"))

feature_df_rated = feature_df.filter(feature_df.rating == 1).groupBy("userID", "paperID", "rating", "pages", "year", "type", "libSize").agg(f.avg("pages").over(window).alias("avgPages")).orderBy("userID")

feature_df_unrated = feature_df.filter(feature_df.rating == 0)

# feature_df_unrated = feature_df_unrated.withColumn('avgPages', lit(" ").cast(StringType()))
# feature_df_unrated = feature_df_unrated.select("userID", "paperID", "rating", "pages", "year", "type", "libSize", "avgPages")

# model_df = feature_df_rated.union(feature_df_unrated)

# # model_df = model_df.withColumn("featureVector", f.concat(col("pages"), lit(", "), col("year"), lit(", "), col("type"), lit(", "), col("libSize"), lit(", "), col("avgPages")))
# # model_df = model_df.drop("pages", "year","type", "libSize", "avgPages")

# model_df.where(model_df.userID=="1eac022a97d683eace8815545ce3153f").show(truncate=False)

+--------------------+-------+---------+------+---------+
|              userID|libSize|   papers|rating|  unrated|
+--------------------+-------+---------+------+---------+
|01ecfc66d01b0dd84...|      6| [970908]|     1|[8240584]|
|01ecfc66d01b0dd84...|      6|  [84168]|     1|  [11511]|
|01ecfc66d01b0dd84...|      6|[1843498]|     1|[1121661]|
|01ecfc66d01b0dd84...|      6| [808689]|     1|[2625223]|
|01ecfc66d01b0dd84...|      6| [508086]|     1|[3050075]|
|01ecfc66d01b0dd84...|      6| [397003]|     1|[3464748]|
|0ad6516296d95068c...|     11|[2734645]|     1|[8240584]|
|0ad6516296d95068c...|     11|[1218426]|     1|  [11511]|
|0ad6516296d95068c...|     11| [748113]|     1|[1121661]|
|0ad6516296d95068c...|     11| [179912]|     1|[3018437]|
|0ad6516296d95068c...|     11|[2069872]|     1|[3050075]|
|0ad6516296d95068c...|     11| [359059]|     1| [486957]|
|0ad6516296d95068c...|     11| [763543]|     1|[1290791]|
|0ad6516296d95068c...|     11|[2967998]|     1|[1065319]|
|0ad6516296d95

In [64]:
feature_df.where(feature_df.userID=="1eac022a97d683eace8815545ce3153f").show(truncate=False)

+--------+--------------------------------+-------+------+-----------+-----+----+
|paperID |userID                          |libSize|rating|type       |pages|year|
+--------+--------------------------------+-------+------+-----------+-----+----+
|1121661 |1eac022a97d683eace8815545ce3153f|334    |1     |article    |27   |2001|
|238188  |1eac022a97d683eace8815545ce3153f|334    |1     |article    |13   |1997|
|1042553 |1eac022a97d683eace8815545ce3153f|334    |1     |article    |11   |1995|
|1242600 |1eac022a97d683eace8815545ce3153f|334    |1     |article    |4    |2007|
|523772  |1eac022a97d683eace8815545ce3153f|334    |1     |article    |null |2005|
|531300  |1eac022a97d683eace8815545ce3153f|334    |1     |article    |5    |1953|
|2492402 |1eac022a97d683eace8815545ce3153f|334    |1     |article    |1    |2008|
|255030  |1eac022a97d683eace8815545ce3153f|334    |1     |article    |7    |2000|
|3010240 |1eac022a97d683eace8815545ce3153f|334    |1     |article    |null |2008|
|2058201 |1eac02